# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [21]:
# Import relevant library
library('tidyverse')

# Import data
setwd('/Users/jenahblack/Documents/Current_Coursework/Data_Science/DataSciencePsychNeuro_CMU85732/Homeworks/lexDat')
lexdata_original <- read.csv('LexicalData.csv')
head(lexdata_original)
items_original <- read.csv('Items.csv')
head(items_original)

# convert the reaction times to numeric
lexdata <- lexdata_original %>% 
  filter(!is.na(D_RT)) %>%
  mutate(D_RT = gsub(',', '', D_RT),
         D_RT = as.numeric(D_RT))
items <- items_original %>%
  select(Word, Length, Log_Freq_HAL)

dat <- left_join(lexdata, items, by=c("D_Word"="Word"))


,Sub_ID,Trial,Type,D_RT,D_Word,Outlier,D_Zscore
,<int>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,"1,094",refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


,Occurrences,Word,Length,Freq_HAL,Log_Freq_HAL
,<int>,<chr>,<int>,<chr>,<dbl>
1,1,synergistic,11,284,5.649
2,1,synonymous,10,951,6.858
3,1,syntactical,11,114,4.736
4,1,synthesis,9,"6,742",8.816
5,1,synthesized,11,"2,709",7.904
6,1,synthesizer,11,"1,390",7.237


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [19]:
model1 <- lm(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL*Length, data = dat)
summary(model1)


Call:
lm(formula = D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL * Length, 
    data = dat)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


Now, install `lme4` using `install.packages()` and then load the library. 

In [5]:
install.packages("lme4")
library(lme4)


The downloaded binary packages are in
	/var/folders/cr/5w4bx6nn5tbc9spb5wzmmk5w0000gn/T//RtmppfZAEZ/downloaded_packages


Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [23]:
model2 <- lmer(D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL*Length + (1|Sub_ID), data = dat)
summary(model2)

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL + Length + Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: dat

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

> The t-values are larger in the mixed effects model, due to the fact that the standard errors are smaller. This is because the coefficients can be better estimates of the effect when the subjects have random slopes.

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [13]:
AIC(model1)
AIC(model2)

AIC(model1) > AIC(model2)


[1] 914591.2

[1] 888475.9

[1] TRUE

> The AIC for mixed effects model is lower, indicating the mixed effects model is a better fit for these data. 
> 

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

> Assuming that the effect of interest is how the length of the letter strings impact reaction time, you could include a random intercept for the trial number. There may be practice effects (such that reaction times are generally faster later in the task), or maybe the subjects could get tired and slow down.
> 

**DUE:** 5pm EST, March 25, 2021

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> Nope